In [1]:
from  geopy.geocoders import Nominatim
import pandas as pd
import numpy as np

import time

from ratelimit import limits, sleep_and_retry

In [2]:
df = pd.read_excel("Direccionesunicas.xlsx")

#Agregar colunas para guardar la latitud y longitud.
df["Latitud"]=0.00
df["Longitud"]=0.00

In [3]:
df.tail()

,Direccion,0,Latitud,Longitud
191,"Villa Morada, Pantoja",12,0.0,0.0
192,"Villas Agrícolas, Distrito Nacional",964,0.0,0.0
193,"Vista Verde, Pantoja",20,0.0,0.0
194,"Yacó, La Guáyiga",12,0.0,0.0
195,"Zona Industrial Herrera, Santo Domingo Oeste",68,0.0,0.0


In [11]:
#Buscar las latitudes que no estan procesadas

dfPend = df[df.Latitud==0]
dfPend.count()

Direccion    61
0            61
Latitud      61
Longitud     61
dtype: int64

In [3]:
geolocator = Nominatim(user_agent="tfm-geocoding-python")

C:\Users\Enel\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [6]:
@sleep_and_retry
@limits(calls=1,period=5)
def rate_limited_geocode(query):
    loc =geolocator.geocode(query)
    
    if (loc is None):
        loc = type('loc', (object,), {'latitude':np.nan,'longitude':np.nan})()
        
    return loc


In [7]:
print("Start Time : %s" % time.ctime())

Start Time : Fri Sep 20 17:22:57 2019


In [8]:
#Recorrer el dataset con las direcciones unicas
#IMPORTANTE para la libreria NONATIM: No heavy uses (an absolute maximum of 1 request per second).

for index, row in dfPend.iterrows():
    loc = rate_limited_geocode(dfPend.at[index,"Direccion"] ) 
    dfPend.at[index,"Latitud"] =loc.latitude
    dfPend.at[index,"Longitud"] =loc.longitude

GeocoderTimedOut: Service timed out

In [ ]:
print ("End Time: %s" % time.ctime())

In [9]:
dfPend.head()

,Direccion,0,Latitud,Longitud
0,"24 de Abril, Distrito Nacional",192,18.505528,-69.895642
1,"30 de Mayo, Distrito Nacional",188,18.441060,-69.936756
2,"Alameda, Santo Domingo Oeste",176,18.494496,-69.995485
3,"Alma Rosa, Santo Domingo Este",932,18.491873,-69.855955
4,"Alto de Chavón, Los Alcarrizos",60,NaN,NaN


In [10]:
df.update(dfPend)

In [12]:
df.to_excel("Locations-listas.xlsx")